# Text Concreteness (R)

## Create `dfm`

In [ ]:
library(spacyr)
library(quanteda)

spacy_initialize(model='en_core_web_lg',
                 save_profile = T)

ld(pjson, ldtype='feather')

In [17]:
# create corpus
corpus = pjson[, .(pid, project_desc)] %>%
    corpus(docid_field='pid', text_field='project_desc')

In [ ]:
# tokenize with spacy
# the results is a data.frame
# 1) keep both "raw" and "lemma" tokens 
# 2) tokens are case-sensitive
tokens_as_df = corpus %>%
    spacy_parse(pos=F, entity=F)

sv(tokens_as_df)
sv()

In [144]:
# convert `tokens_as_df` to quanteda `tokens` object
# 1) we use the lemmatized tokens, because the lookup table is also lemmatized 
# 2) tokens are case-sensitive
tokens_as_qeda = tokens_as_df %>%
    as.tokens(use_lemma=T)

tokens_as_qeda[1]

sv(tokens_as_qeda)

Tokens consisting of 1 document.
1000064918 :
 [1] "the"     "Beard"   "be"      "a"       "comedy"  "base"    "comic"  
 [8] "about"   "an"      "average" "guy"     "that"   
[ ... and 108 more ]
-tokens_lemmatized- saved  (10.24 secs)


In [ ]:
# get the number of tokens of each doc
ntoken_corpus = ntoken(tokens_as_qeda)
ntoken_corpus = data.table(pid=names(ntoken_corpus), ntoken=ntoken_corpus)
ntoken_corpus[1]
sv(ntoken_corpus)

In [33]:
# Convert tokens to dfm
ld(tokens_as_qeda, force=T)

tokens_to_dfm <- function(tokens_as_qeda, startpos=1, endpos=-1) {
    # select tokens
    tokens = tokens_select(tokens_as_qeda, startpos=startpos, endpos=endpos)
    
    # create ngram
    tokens_ngram = tokens %>%
        tokens_ngrams(n=1:2, concatenator = " ")
    
    # create dfm
    cs_dfm = tokens_ngram %>% dfm(tolower=T, stem=F)
}

cs_dfm = tokens_to_dfm(tokens_as_qeda)
cs_dfm_first200 = tokens_to_dfm(tokens_as_qeda, endpos=200)

# sv(cs_dfm)
sv(cs_dfm_first200)

tokens_as_qeda (47.5 MB) already loaded, will NOT load again! (0 secs) (2021-03-07 5:55 PM)
"cs_dfm_first200" saved as "cs_dfm_first200.rds" (43.1 MB) (7.55 secs, 2021-03-07 17:56:10)


## Compute B-score

In [4]:
# get stopwords from nltk (Python code)
import nltk
import pandas as pd
from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))
df = pd.DataFrame({'word':list(stopwords)})
sv('df', svname='nltk_stopwords')

"df" saved as "nltk_stopwords.feather" (2.5 KB) (<1s)


In [1]:
# ------------ Create bscore dict ----------------
# load stop word list
ld(nltk_stopwords, force=T)
nltk_stopwords = nltk_stopwords[,word]

# read raw bscore
bscore_dt = fread('/home/yu/OneDrive/Construal/data/concreteness score.csv')[, .(word=str_trim(Word), score=Conc.M)]

# create TWO bscore, one has stopwords, one doesn't 
bscore = bscore_dt$score
names(bscore) = bscore_dt$word

bscore_nostopwords = bscore[!(names(bscore)%in%nltk_stopwords)]

sprintf('%s%% words are stopwords', (1-length(bscore_nostopwords)/length(bscore))*100%>%round(2)) %>% cat()

"nltk_stopwords.feather" (2.5 KB) loaded (0.02 secs) (2021-05-06 8:37 AM)
0.310356910447018% words are stopwords

In [58]:
# ------------ Create bscore from dtm ----------------

ld(ntoken_corpus)
ld(cs_dfm, force=T)
ld(cs_dfm_first200, force=T)
ld(bscore_nostopwords, force=T)


dfm_to_bscore <- function(cs_dfm, bscore_dict, type_name='') {
    ntoken_name = str_c('ntoken_bscore', type_name)
    ntoken_unique_name = str_c('ntoken_unique', type_name)
    ntoken_bscore_unique_name = str_c('ntoken_bscore_unique', type_name)
    bscore_name = str_c('bscore', type_name)
    
    output_name = c('pid', bscore_name)

    ntoken_unique = ntype(cs_dfm)

    dfm_bscore = dfm_match(cs_dfm, names(bscore_dict))
    ntoken_bscore = ntoken(dfm_bscore)
    ntoken_bscore_unique = ntype(dfm_bscore) 
    ntoken_bscore_dt = data.table(pid=names(ntoken_bscore))
    ntoken_bscore_dt = ntoken_bscore_dt[, (ntoken_name) := ntoken_bscore
        ][, (ntoken_unique_name) := ntoken_unique
        ][, (ntoken_bscore_unique_name) := ntoken_bscore_unique]
    
    
    dfm_bscore_weighted = dfm_weight(dfm_bscore, weights=bscore_dict)
    dfm_bscore_weighted = convert(dfm_bscore_weighted, 'data.frame',
                                  docid_field='pid'
                                 ) %>% as.data.table()
    
    bscore_by_pid = dfm_bscore_weighted[, (bscore_name) := rowSums(.SD),
                                        .SDcols=is.numeric
        ][, ..output_name]
    
    bscore = bscore_by_pid[ntoken_bscore_dt, on=.(pid)]
}

bscore_bypid = dfm_to_bscore(cs_dfm, bscore)
# bscore_bypid_nostopwords = dfm_to_bscore(cs_dfm, bscore_nostopwords, 
#                                          type_name='_nostopwords')
# bscore_bypid_firstn = dfm_to_bscore(cs_dfm_first200, bscore,
#                                     type_name='_first200')
# bscore_bypid_firstn_nostopwords = dfm_to_bscore(cs_dfm_first200,
#                                                 bscore_nostopwords,
#                                                 type_name='_first200_nostopwords')

ntoken_corpus (282.9 KB) already loaded, will NOT load again! (0 secs) (2021-05-06 9:34 AM)
"cs_dfm.rds" (116.8 MB) loaded (3.99 secs) (2021-05-06 9:34 AM)
"cs_dfm_first200.rds" (43.1 MB) loaded (1.71 secs) (2021-05-06 9:34 AM)
"bscore_nostopwords.rds" (232.4 KB) loaded (0.03 secs) (2021-05-06 9:34 AM)


In [65]:
fwrite(bscore_bypid_final, '../data/bscore_bypid.csv')

In [61]:
bscore_bypid_final = bscore_bypid[bscore_bypid_nostopwords, on=.(pid)
    ][bscore_bypid_firstn, on=.(pid)
    ][bscore_bypid_firstn_nostopwords, on=.(pid)
    ][ntoken_corpus, on=.(pid)]

sv(bscore_bypid_final, 'bscore_bypid')
fwrite(bscore_bypid_final, '../data/bscore_bypid.csv')

"bscore_bypid_final" saved as "bscore_bypid.feather" (2.4 MB) (0.01 secs, 2021-05-06 09:35:38)


In [64]:
bscore_bypid_final %>% names()

[1] "pid"                                      
 [2] "bscore"                                   
 [3] "ntoken_bscore"                            
 [4] "ntoken_unique"                            
 [5] "ntoken_bscore_unique"                     
 [6] "bscore_nostopwords"                       
 [7] "ntoken_bscore_nostopwords"                
 [8] "ntoken_unique_nostopwords"                
 [9] "ntoken_bscore_unique_nostopwords"         
[10] "bscore_first200"                          
[11] "ntoken_bscore_first200"                   
[12] "ntoken_unique_first200"                   
[13] "ntoken_bscore_unique_first200"            
[14] "bscore_first200_nostopwords"              
[15] "ntoken_bscore_first200_nostopwords"       
[16] "ntoken_unique_first200_nostopwords"       
[17] "ntoken_bscore_unique_first200_nostopwords"
[18] "ntoken"

# Word Freq (R)

## Compute freq_dict

Compute the word frequency from the Google dataset

In [110]:
ld(tokens_as_df)
ld(nltk_stopwords)

google_freqdict_withstop = fread('data/freqdict.csv')

tokens_as_df (462.3 MB) already loaded, will NOT load again! (0 secs) (2021-04-06 7:35 PM)
nltk_stopwords (2.5 KB) already loaded, will NOT load again! (0 secs) (2021-04-06 7:35 PM)


In [135]:
google_freqdict = google_freqdict_withstop[
      !(word %in% nltk_stopwords$word),
    ][, .(word, freq_google_withoutstop=freq/max(freq))
    ][google_freqdict_withstop[, .(word, freq_google_withstop=freq)], 
      on=.(word)
    # ][, ':='(freq_google_withoutstop=nafill(freq_google_withoutstop, 'const', 0))
    ][order(-freq_google_withstop)]

google_freqdict[1:2]

word,freq_google_withoutstop,freq_google_withstop
<chr>,<dbl>,<dbl>
the,NA,1.0000000
of,NA,0.5684659


In [169]:
sv(google_freqdict)
fwrite(google_freqdict, 'data/Sharing/google_freqdict.csv')

"google_freqdict" saved as "google_freqdict.feather" (1.2 MB) (0.01 secs, 2021-04-06 23:43:00)


Compute the word frequency from the Kickstarer dataset

In [136]:
punct = c(',', '.', '-', '?', '!', '(', ')', '$', '/', ':', ' ', '"', intToUtf8(160))

kck_freqdict_withstopwords = tokens_as_df[, .(doc_id, word=tolower(token))
    ][!(word %in% punct)
    ][, .(n=.N), keyby=.(word)
    ][, ':='(freq_kck_withstop=n/max(n))
    ][order(-freq_kck_withstop), .(word, freq_kck_withstop)]

kck_freqdict_withoutstopwords = tokens_as_df[, .(doc_id, word=tolower(token))
    ][(!(word %in% punct)) & (!(word %in% nltk_stopwords$word))
    ][, .(n=.N), keyby=.(word)
    ][, ':='(freq_kck_withoutstop=n/max(n))
    ][order(-freq_kck_withoutstop), .(word, freq_kck_withoutstop)]

fwrite(kck_freqdict_withstopwords, 'data/Sharing/kck_freqdict_withstopwords.csv')
fwrite(kck_freqdict_withoutstopwords, 'data/Sharing/kck_freqdict_withoutstopwords.csv')

kck_freqdict = kck_freqdict_withoutstopwords[kck_freqdict_withstopwords, on=.(word)
    # ][, ':='(freq_kck_withoutstop=nafill(freq_kck_withoutstop, 'const', 0))
    ][order(-freq_kck_withstop)]

In [ ]:
ld(kck_freqdict)
kck_freqdict

In [ ]:
fwrite(kck_freqdict, '../data/word_freq_kick.csv')

In [168]:
sv(kck_freqdict)
fwrite(kck_freqdict, 'data/Sharing/kck_freqdict.csv')

"kck_freqdict" saved as "kck_freqdict.feather" (9 MB) (0.35 secs, 2021-04-06 23:38:29)


In [ ]:
kck_freqdict

Merge Kickstar_freq with Google_freq

In [153]:
freq_dict = google_freqdict[kck_freqdict, on=.(word)]
freq_dict = freq_dict[order(-freq_google_withstop)
    ][, ':='(
      top_google_withoutstop=word %in% word[!is.na(freq_google_withoutstop)][1:5000],
      top_google_withstop=word %in% word[!is.na(freq_google_withstop)][1:5000]
      )
    ][order(-freq_kck_withstop)
    ][, ':='(
      top_kck_withoutstop=word %in% word[!is.na(freq_kck_withoutstop)][1:5000],
      top_kck_withstop=word %in% word[!is.na(freq_kck_withstop)][1:5000]
      )]
      
sv(freq_dict)

"freq_dict" saved as "freq_dict.feather" (9.8 MB) (0.35 secs, 2021-04-06 20:09:14)


## Compute score

Compute the "frequency score"

- top 5000 words or all
- with/without stopwords
- based on kickstarter or LVIS

So there're 2 * 2 * 2=8 versions

In [155]:
ld(freq_dict, force=T)
freq_dict[1]

"freq_dict.feather" (9.8 MB) loaded (0.19 secs) (2021-04-06 8:09 PM)


word,freq_google_withoutstop,freq_google_withstop,freq_kck_withoutstop,freq_kck_withstop,top_google_withoutstop,top_google_withstop,top_kck_withoutstop,top_kck_withstop
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>
the,NA,1,NA,1,FALSE,TRUE,FALSE,TRUE


In [ ]:
punct = c(',', '.', '-', '?', '!', '(', ')', '$', '/', ':', ' ', '"', intToUtf8(160))

score = tokens_as_df[,.(doc_id, word=tolower(token))
    ][!(word %in% punct)
    ][freq_dict, 
      on=.(word), nomatch=NULL
    ][, .(score_google_all_withstop=sum(freq_google_withstop, na.rm=T),
          score_google_all_withoutstop=sum(freq_google_withoutstop, na.rm=T),
          score_google_top5000_withstop=sum(freq_google_withstop[top_google_withstop], na.rm=T),
          score_google_top5000_withoutstop=sum(freq_google_withoutstop[top_google_withstop], na.rm=T),
          score_kck_all_withstop=sum(freq_kck_withstop, na.rm=T),
          score_kck_all_withoutstop=sum(freq_kck_withoutstop, na.rm=T),
          score_kck_top5000_withstop=sum(freq_kck_withstop[top_google_withstop], na.rm=T),
          score_kck_top5000_withoutstop=sum(freq_kck_withoutstop[top_google_withstop], na.rm=T),
          n_words_withstop=.N,
          n_words_withoutstop=sum(!(word%in%(nltk_stopwords$word)))
        ),
      keyby=.(doc_id)]

In [164]:
sv(score)
fwrite(score, 'data/Sharing/text_freq_score.csv')

"score" saved as "score.feather" (3.3 MB) (0.01 secs, 2021-04-06 23:32:40)


In [1]:
ld(score)

"score.feather" (3.3 MB) loaded (0.04 secs) (2021-05-06 8:05 AM)


In [2]:
score %>% names()

[1] "doc_id"                           "score_google_all_withstop"       
 [3] "score_google_all_withoutstop"     "score_google_top5000_withstop"   
 [5] "score_google_top5000_withoutstop" "score_kck_all_withstop"          
 [7] "score_kck_all_withoutstop"        "score_kck_top5000_withstop"      
 [9] "score_kck_top5000_withoutstop"    "n_words_withstop"                
[11] "n_words_withoutstop"

# Text